In [1]:
import pysam
from collections import Counter
import gzip
import os
import random

In [20]:
def get_tags_from_bam(bamfile_path, gene_tag, UMI_tag,sub_props):
    # List to store tags
    gene_count_dict = {sub_prop: {} for sub_prop in sub_props}
    res_cnt = {sub_prop: Counter() for sub_prop in sub_props}
    # Open the BAM file
    idx = 0
    with pysam.AlignmentFile(bamfile_path, "rb",require_index=False) as bamfile:
        # Iterate over each read
        for read in bamfile:
            idx += 1
            if read.has_tag("GE"):
                gene = read.get_tag(gene_tag, with_value_type=False)
            else: gene = None
            cx_value = int(read.get_tag("Cx"))
            cy_value = int(read.get_tag("Cy"))
            bc = f"{cx_value}_{cy_value}"

            UMI = read.get_tag(UMI_tag, with_value_type=False)
            for sub_prop in sub_props:
                random_num = random.randint(0, 99)
                if random_num < sub_prop * 100:
                    # Check if the tag is present in the read
                    if (gene is not None) and (bc is not None) and (UMI is not None):
                        # Add the tag value to the list
                        if gene not in gene_count_dict[sub_prop]:
                            gene_count_dict[sub_prop][gene] = {}
                        gene_count_dict[sub_prop][gene].setdefault(bc, set()).add(UMI)
                        res_cnt[sub_prop]["transcriptomic"] += 1 
                    
            if idx % 1000000 ==0:
                print(f"Total reads processed: {idx}")
                print(res_cnt[sub_prop])
        for sub_prop in sub_props:
            for gene in gene_count_dict[sub_prop]:
                gene_count_dict[sub_prop][gene] = {bc: len(gene_count_dict[sub_prop][gene][bc]) for bc in gene_count_dict[sub_prop][gene]}

    return gene_count_dict,res_cnt

In [ ]:
def write_mtx(gene_count_dict, out_dir):
    """
    gene_count_dict: dict in dict {gene_id: {barcode: UMI_count}, ... }
    """
    gene_list = []  # Create an empty list to store gene IDs
    bc_list = []    # Create an empty list to store barcode IDs
    
    # Iterate through the gene_count_dict to collect gene and barcode IDs
    for ge in gene_count_dict:
        gene_list.append(ge)
        for bc in gene_count_dict[ge]:
            bc_list.append(bc)
    
    total_cnt = len(bc_list)  # Calculate the total number of unique barcodes
    
    # Remove duplicate barcodes by converting the list to a set and back to a list
    bc_list = list(set(bc_list))
    
    # Create a dictionary to map gene IDs to their index (1-based indexing)
    probe_dict = dict([(pid, ix + 1) for ix, pid in enumerate(gene_list)])  # 1-based indexing
    
    # Create a dictionary to map barcode IDs to their index (1-based indexing)
    bc_dict = dict([(bc, ix + 1) for ix, bc in enumerate(bc_list)])  # 1-based indexing
    
    # Open and write the gene features file
    out_feature = gzip.open(os.path.join(out_dir, "digital_expression_features.tsv.gz"), "wb")
    for ge in gene_list:
        out_feature.write(ge.encode('utf-8') + b'\n')  # Write gene IDs to the file
    out_feature.close()
    
    # Open and write the barcode file
    out_barcode = gzip.open(os.path.join(out_dir, "digital_expression_barcodes.tsv.gz"), "wb")
    for bc in bc_list:
        out_barcode.write(bytes(bc + '\n', "UTF-8"))  # Write barcode IDs to the file
    out_barcode.close()
    
    # Open and write the matrix file in MatrixMarket format
    out_mtx = gzip.open(os.path.join(out_dir, "digital_expression_matrix.mtx.gz"), "wb")
    out_mtx.write(b'%%MatrixMarket matrix coordinate integer general\n%\n')  # Write MatrixMarket headers
    out_mtx.write(bytes("{}\t{}\t{}\n".format(len(gene_list), len(bc_list), total_cnt), "UTF-8" ))  # Write matrix dimensions
    for ge in gene_list:  # Iterate over genes
        for bc in gene_count_dict[ge]:  # Iterate over barcodes for each gene
            out_mtx.write(bytes("{}\t{}\t{}\n".format(probe_dict[ge], bc_dict[bc], gene_count_dict[ge][bc]), "UTF-8"))  # Write matrix data
    out_mtx.close()

In [21]:
if __name__ == "__main__":
    gene_tag = "GE"
    UMI_tag = "UR"
    bamfile_path = "mouse_brain_B02223D2.bam"  # replace with your bam file path
    gene_count_dict,res_cnt = get_tags_from_bam(bamfile_path, gene_tag, UMI_tag, sub_props1)

[E::idx_find_and_load] Could not retrieve index file for 'mouse_brain_B02223D2.bam'


Total reads processed: 1000000
Counter({'transcriptomic': 65267})
Total reads processed: 2000000
Counter({'transcriptomic': 144459})
Total reads processed: 3000000
Counter({'transcriptomic': 234968})
Total reads processed: 4000000
Counter({'transcriptomic': 283175})
Total reads processed: 5000000
Counter({'transcriptomic': 333009})
Total reads processed: 6000000
Counter({'transcriptomic': 403457})
Total reads processed: 7000000
Counter({'transcriptomic': 493057})
Total reads processed: 8000000
Counter({'transcriptomic': 565598})
Total reads processed: 9000000
Counter({'transcriptomic': 650244})
Total reads processed: 10000000
Counter({'transcriptomic': 650244})
Total reads processed: 11000000
Counter({'transcriptomic': 650244})
Total reads processed: 12000000
Counter({'transcriptomic': 650244})
Total reads processed: 13000000
Counter({'transcriptomic': 650244})
Total reads processed: 14000000
Counter({'transcriptomic': 650244})
Total reads processed: 15000000
Counter({'transcriptomic':

In [23]:
from scipy.io import mmwrite

In [26]:
for prop in sub_props1:
    out_dir = f"./total_downsampled/{prop}"
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    write_mtx(gene_count_dict[prop],out_dir)

['21102_8039', '10017_8562', '7175_19258', '12773_12980', '15364_5138', '10399_20673', '17339_8464', '18751_9227', '12123_13006', '12059_16818']
['Xkr4', 'Gm1992', 'Gm37381', 'Rp1', 'Mrpl15', 'Sox17', 'Lypla1', 'Rgs20', 'Gm37988', 'Tcea1']
